In [1]:
import os

import pandas as pd
import numpy as np
import sklearn

In [2]:
root_path = os.getcwd()
data_path = root_path + '/data/exoplanets.csv'

In [3]:
df = pd.read_csv(data_path)
df.head()

,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,K00753.01,NaN,CANDIDATE,CANDIDATE,0.000,0,0,0,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [37]:
df.shape

(13450, 46)

# Preprocessing

First let's take a look at missing values in our columns

In [4]:
missing_values = df.isna().sum() / len(df) * 100
missing_values.sort_values(ascending=False)[:10]

koi_teq_err2      100.000000
koi_teq_err1      100.000000
kepler_name        75.334588
koi_score          15.788373
koi_steff_err2      5.050188
koi_srad_err2       4.893350
koi_srad_err1       4.893350
koi_slogg_err2      4.893350
koi_slogg_err1      4.893350
koi_steff_err1      4.893350
dtype: float64

We decide we will drop those columns with more than 20% of missing values. The other we will impute???

In [5]:
df = df.drop(columns={'koi_teq_err2','koi_teq_err1','kepler_name'})
df.columns

Index(['kepid', 'kepoi_name', 'koi_disposition', 'koi_pdisposition',
       'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_tce_delivname', 'koi_steff',
       'koi_steff_err1', 'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1',
       'koi_slogg_err2', 'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra',
       'dec', 'koi_kepmag'],
      dtype='object')

Now we look into the 

In [22]:
numerical_columns = []
categorical_columns = []

for column in df.columns:
    if pd.api.types.is_numeric_dtype(df[column]):
        numerical_columns.append(column)
    else:
        categorical_columns.append(column)
categorical_columns.append('kepid')

df_num = df[numerical_columns]
df_cat = df[categorical_columns]

df_num.shape, df_cat.shape

((9564, 42), (9564, 5))

Now we perform imputation on those 

In [23]:
df_num.head()

,kepid,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,10797460,1.000,0,0,0,0,9.488036,2.780000e-05,-2.780000e-05,170.538750,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,10797460,0.969,0,0,0,0,54.418383,2.480000e-04,-2.480000e-04,162.513840,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,0.000,0,0,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,1.000,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [28]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
df_filled_num = pd.DataFrame(imputer.fit_transform(df_num),columns=df_num.columns)

In [34]:
df_filled_num.isna().sum()

kepid                0
koi_score            0
koi_fpflag_nt        0
koi_fpflag_ss        0
koi_fpflag_co        0
koi_fpflag_ec        0
koi_period           0
koi_period_err1      0
koi_period_err2      0
koi_time0bk          0
koi_time0bk_err1     0
koi_time0bk_err2     0
koi_impact           0
koi_impact_err1      0
koi_impact_err2      0
koi_duration         0
koi_duration_err1    0
koi_duration_err2    0
koi_depth            0
koi_depth_err1       0
koi_depth_err2       0
koi_prad             0
koi_prad_err1        0
koi_prad_err2        0
koi_teq              0
koi_insol            0
koi_insol_err1       0
koi_insol_err2       0
koi_model_snr        0
koi_tce_plnt_num     0
koi_steff            0
koi_steff_err1       0
koi_steff_err2       0
koi_slogg            0
koi_slogg_err1       0
koi_slogg_err2       0
koi_srad             0
koi_srad_err1        0
koi_srad_err2        0
ra                   0
dec                  0
koi_kepmag           0
dtype: int64

In [29]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
df_filled_cat = pd.DataFrame(imputer.fit_transform(df_cat),columns=df_cat.columns)

In [33]:
df_filled_cat.isna().sum()

kepoi_name           0
koi_disposition      0
koi_pdisposition     0
koi_tce_delivname    0
kepid                0
dtype: int64

In [35]:
df = pd.merge(df_filled_num,df_filled_cat,on='kepid')
df.shape

(13450, 46)

In [36]:
df.head()

,kepid,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,...,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,kepoi_name,koi_disposition,koi_pdisposition,koi_tce_delivname
0,10797460.0,1.000,0.0,0.0,0.0,0.0,9.488036,0.000028,-0.000028,170.538750,...,0.927,0.105,-0.061,291.93423,48.141651,15.347,K00752.01,CONFIRMED,CANDIDATE,q1_q17_dr25_tce
1,10797460.0,1.000,0.0,0.0,0.0,0.0,9.488036,0.000028,-0.000028,170.538750,...,0.927,0.105,-0.061,291.93423,48.141651,15.347,K00752.02,CONFIRMED,CANDIDATE,q1_q17_dr25_tce
2,10797460.0,0.969,0.0,0.0,0.0,0.0,54.418383,0.000248,-0.000248,162.513840,...,0.927,0.105,-0.061,291.93423,48.141651,15.347,K00752.01,CONFIRMED,CANDIDATE,q1_q17_dr25_tce
3,10797460.0,0.969,0.0,0.0,0.0,0.0,54.418383,0.000248,-0.000248,162.513840,...,0.927,0.105,-0.061,291.93423,48.141651,15.347,K00752.02,CONFIRMED,CANDIDATE,q1_q17_dr25_tce
4,10811496.0,0.000,0.0,0.0,0.0,0.0,19.899140,0.000015,-0.000015,175.850252,...,0.868,0.233,-0.078,297.00482,48.134129,15.436,K00753.01,CANDIDATE,CANDIDATE,q1_q17_dr25_tce
